In [1]:
from langchain_core.output_parsers import StrOutputParser
from PyPDF2 import PdfReader
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import SentenceTransformerEmbeddings
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from typing import List
import os
import getpass
from dotenv import load_dotenv
load_dotenv()

/home/mahindra/.pyenv/versions/my-recommender-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
api_key = os.getenv("GROQ_API_KEY")
api_key

'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'

## Load LLM

In [3]:
def load_llm(id_model, temperature):
    llm = ChatGroq(
        model=id_model,
        temperature=temperature,
        api_key=api_key,
        max_tokens=1024,
        timeout=None,
        max_retries=3
    )
    return llm

llm = load_llm('meta-llama/llama-4-maverick-17b-128e-instruct', 0.3)
#openai/gpt-oss-safeguard-20b
#meta-llama/llama-4-maverick-17b-128e-instruct

menggunakan model ini karena terbaru dan sangat mendukung integrasi Agent AI sehingga menghasilkan output yang akurat

In [4]:
# document laoder
def extract_text_from_pdf(file_path :str) -> List[Document]:
    try : 
        
        reader= PyMuPDFLoader(file_path)
        docs = reader.load()
        print(f"✅ Berhasil memproses Dokumen")
        return docs
    except Exception as e:
        print(f"Error saat memproses PDF : {str(e)}")
        return []
#chunking text
def chunk_text(documents : List[Document],chunk_size=1000,overlap=200) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size= chunk_size,
        chunk_overlap=overlap,
        length_function=len,
        separators=["\n\n", "\n", '\n●','\n1','\n2','\n3','\n4','\n5','\n6','\n7','\n8','\n9','\n10'," "]
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Chunking selesai. Total halaman asli : {len(documents)}")
    return chunks

#index to qdrant
def index_to_qdrant(chunks : List[Document], url : str , collection_name : str)-> None:
    if not chunks:
        print("❌ Tidak ada chunks yang valid untuk di-index.")
        return

    try:
        # Inisialisasi model embedding yang akan digunakan
        model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

        print(f"⏳ Memulai indexing ke Qdrant Collection: {collection_name}...")
        
        # 2. Indexing Otomatis
        # Qdrant.from_documents secara otomatis menangani:
        # a) Mengubah chunks menjadi embeddings.
        # b) Menyimpan embeddings, teks, dan metadata ke Qdrant.
        Qdrant.from_documents(
            documents=chunks,
            embedding=model,
            url = url,
            collection_name=collection_name,
        )
        
        print("✅ Indexing ke Qdrant Selesai!")

    except Exception as e:
        print(f"❌ Gagal Indexing ke Qdrant. Cek koneksi atau API Key Anda. Error: {e}")



In [5]:
# pdf_path = 'Handbook_BNI_Mbank.pdf'
qdrant_url = "http://localhost:6333"
collection_name = 'user_manual_BNI_Mbank'

folder= "./static"
for filename in os.listdir(folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder,filename)   
        text_book = extract_text_from_pdf(pdf_path)
        chunk = chunk_text(text_book)

        #save qdrant
        index_to_qdrant(chunk,qdrant_url,collection_name)


✅ Berhasil memproses Dokumen
Chunking selesai. Total halaman asli : 6


/tmp/ipykernel_9874/2896873520.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


⏳ Memulai indexing ke Qdrant Collection: user_manual_BNI_Mbank...
✅ Indexing ke Qdrant Selesai!
✅ Berhasil memproses Dokumen
Chunking selesai. Total halaman asli : 31
⏳ Memulai indexing ke Qdrant Collection: user_manual_BNI_Mbank...
✅ Indexing ke Qdrant Selesai!
✅ Berhasil memproses Dokumen
Chunking selesai. Total halaman asli : 2
⏳ Memulai indexing ke Qdrant Collection: user_manual_BNI_Mbank...
✅ Indexing ke Qdrant Selesai!
✅ Berhasil memproses Dokumen
Chunking selesai. Total halaman asli : 5
⏳ Memulai indexing ke Qdrant Collection: user_manual_BNI_Mbank...
✅ Indexing ke Qdrant Selesai!
✅ Berhasil memproses Dokumen
Chunking selesai. Total halaman asli : 5
⏳ Memulai indexing ke Qdrant Collection: user_manual_BNI_Mbank...
✅ Indexing ke Qdrant Selesai!


In [6]:
from qdrant_client import QdrantClient
from qdrant_client.http.exceptions import UnexpectedResponse

# Ganti 'localhost' dan 6333 dengan alamat server Qdrant Anda
QDRANT_HOST = "localhost"
QDRANT_PORT = 6333 # Port REST API

try:
    # Buat instance klien
    client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT)
    print(f"Mencoba koneksi ke: {QDRANT_HOST}:{QDRANT_PORT}")
    
    # Tambahkan timeout untuk mencegah pemblokiran yang terlalu lama (opsional, default 5 detik)
    # client = QdrantClient(host=QDRANT_HOST, port=QDRANT_PORT, timeout=5)

    # Lakukan operasi ringan yang memerlukan koneksi
    info = client.get_collections() 
    
    ## Alternatif: Cek informasi server (lebih mendalam)
    # info = client.get_locks() 
    # info = client.info()
    
    # Jika berhasil, cetak hasilnya
    print("✅ KONEKSI BERHASIL!")
    print(f"Informasi Koleksi Qdrant: {info.collections}")

except ConnectionRefusedError:
    print(f"❌ KONEKSI GAGAL: Server Qdrant tidak berjalan atau menolak koneksi pada {QDRANT_HOST}:{QDRANT_PORT}")
except UnexpectedResponse as e:
    # Terjadi jika server berjalan tetapi merespons dengan kode status error
    print(f"❌ KONEKSI GAGAL: Server merespons dengan Error: {e}")
except Exception as e:
    print(f"❌ KONEKSI GAGAL: Terjadi kesalahan lain: {e}")

Mencoba koneksi ke: localhost:6333
✅ KONEKSI BERHASIL!
Informasi Koleksi Qdrant: [CollectionDescription(name='book_knowledge'), CollectionDescription(name='user_manual_BNI_Mbank')]


In [7]:
# retriever
def get_qdrant_retriever(url: str = "http://localhost:6333", collection_name: str = None, model_name : str = "all-MiniLM-L6-v2", k : int = 4):
       
    embedding = SentenceTransformerEmbeddings(model_name=model_name)
    client = QdrantClient(url=url)
    
    qdrant = Qdrant(
        client=client,
        collection_name=collection_name, # <== Penentuan COLLECTION
        embeddings=embedding,
    )
    retriever = qdrant.as_retriever(search_kwargs={"k" : k})
    return retriever

In [8]:
retriever = get_qdrant_retriever(collection_name=collection_name)

/tmp/ipykernel_9874/433847635.py:7: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-qdrant package and should be used instead. To use it run `pip install -U `langchain-qdrant` and import as `from `langchain_qdrant import Qdrant``.
  qdrant = Qdrant(


In [9]:
prompt = PromptTemplate(
    input_variables=["context","question"],
    template="""Question: {question}
            Context: {context}
            Please provide a concise and accurate answer based on the context provided. 
            If the context does not contain sufficient information to answer the question, respond with "I don't know."
            """
)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Question: {question}\n            Context: {context}\n            Please provide a concise and accurate answer based on the context provided. \n            If the context does not contain sufficient information to answer the question, respond with "I don\'t know."\n            ')

In [28]:
chain = (
    {"context" :retriever, "question": RunnablePassthrough()}|
    prompt |
    llm | 
    StrOutputParser()
)


In [29]:
print(chain.invoke("Apa aja Fitur baru Whoosh"))

Berdasarkan konteks yang diberikan, fitur baru Whoosh disebutkan dalam dokumen Handbook BNI Mobile Banking. 

Fitur baru Whoosh adalah pemesanan tiket Whoosh (Kereta Cepat Indonesia-China) melalui aplikasi BNI Mobile Banking. Langkah-langkahnya adalah:
1. Pilih Fitur Tiket Whoosh di menu Lifestyle.
2. Lakukan pemesanan hingga pembayaran.
3. Dapatkan tiket keberangkatanmu dalam bentuk QR Code.

Jadi, jawaban atas pertanyaan "Apa aja Fitur baru Whoosh" adalah pemesanan tiket Whoosh melalui BNI Mobile Banking.


In [53]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_classic.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableLambda, RunnableBranch, RunnablePassthrough


In [49]:
context_q_system_prompt = "Kamu adalah asisten AI yang membantu pengguna menjawab pertanyaan tentang BNI Mobile Banking. \
    Gunakan konteks dari dokumen (hasil pencarian retriever) dan riwayat percakapan sebelumnya \
    untuk menjawab dengan akurat, ringkas, dan relevan. \
    Jika konteks tidak mengandung informasi yang cukup untuk menjawab, katakan dengan sopan 'Maaf, saya tidak tahu.'"

context_q_user_prompt = "Pertanyaan: {input}\n\n Konteks:\n{context}"

context_q_prompt = ChatPromptTemplate.from_messages([
    ("system",context_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human",context_q_user_prompt)
])

In [54]:
#Mengecek apakah memiliki konteks
def has_context(inputs) : 
    context = inputs.get("context","")
    return bool(context and context.strip())


history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=context_q_prompt
)



qa_chain = create_stuff_documents_chain(
    llm,
    context_q_prompt,
)

rag_chain = create_retrieval_chain(
    history_aware_retriever,
    qa_chain
)
no_context_chain = RunnableLambda(lambda x : {"answer" : "Maaf saya tidak tahu"})

final_chain = RunnableBranch(
    (has_context,rag_chain),
    RunnablePassthrough() | no_context_chain
)

In [13]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory


store = {}

def get_sessionhistory(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


qa_chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    get_sessionhistory,
    input_messages_key="input",
    history_messages_key="chat_history"
    
)

In [15]:
session_id = "user_42"

In [20]:
response = qa_chain_with_history.invoke({
    "input": "Bagaimana cara menggunakan Mbanking BRI?",
    "context":""
},config={"configurable": {"session_id": session_id}})

In [17]:
response2 = qa_chain_with_history.invoke({
    "input": "Apa pertanyaan terakhir saya?",
    "context":""
},config={"configurable": {"session_id": session_id}})

In [21]:
print(response)

Untuk menggunakan Mbanking BRI, Anda dapat mengikuti langkah-langkah serupa dengan Mbanking BNI, yaitu: 

1. Unduh aplikasi BRIMO (BRI Mobile) melalui Google Play Store atau App Store.
2. Daftar dan aktivasi layanan BRIMO.
3. Login ke aplikasi menggunakan username dan password.
4. Pilih menu yang diinginkan, seperti cek saldo, transfer, atau bayar tagihan.
5. Ikuti instruksi untuk menyelesaikan transaksi.

Namun, saya sarankan untuk memeriksa situs resmi BRI atau menghubungi customer service BRI untuk informasi lebih lanjut dan akurat.


## Diterapkan pada RAG

In [61]:
chat_history = []
input = "Sebutkan Fitur BRI Mobile?"
docs = retriever.invoke(input)
context = "\n\n".join([d.page_content for d in docs]) if docs else ""

chat_history.append(HumanMessage(content=input))

In [62]:
result = rag_chain.invoke({"input":input, "chat_history":chat_history, "context" : context})
chat_history.append(AIMessage(content=result["answer"]))

In [63]:
print(result["answer"])

Maaf, saya tidak bisa menjawab pertanyaan tentang Fitur BRI Mobile karena konteks yang diberikan adalah tentang BNI Mobile Banking. Namun, saya bisa memberikan informasi bahwa BNI Mobile Banking memiliki beberapa fitur seperti:

- Pembayaran dan pembelian Listrik/PLN 
- Informasi saldo 
- Transfer 
- Pembayaran 

Jika Anda ingin tahu lebih lanjut tentang BNI Mobile Banking, saya dapat membantu. Namun, untuk informasi tentang BRI Mobile, saya tidak memiliki data yang cukup. Maaf, saya tidak tahu tentang Fitur BRI Mobile.


In [42]:
check_my_last_question = rag_chain.invoke({"input":"what my last question", "chat_history":chat_history})
print(check_my_last_question["answer"])

InternalServerError: Error code: 503 - {'error': {'message': 'meta-llama/llama-4-maverick-17b-128e-instruct is currently over capacity. Please try again and back off exponentially. Visit https://groqstatus.com to see if there is an active incident.', 'type': 'internal_server_error'}}

In [ ]:
input = "How to perform a transfer payment using BNI Mobile Banking"
chat_history.append(HumanMessage(content=input))

result2 = rag_chain.invoke({"input":input, "chat_history":chat_history})
chat_history.append(AIMessage(content=result2["answer"]))

print(result2["answer"])

To perform a transfer payment using BNI Mobile Banking via BI-Fast, follow these steps:

1. Click on the "Transfer Antar Bank" feature in the Transfer menu.
2. Input the data and select the transfer service type to BI-Fast.
3. Enter your transaction password to complete the transaction.

The transfer can be done 24/7, and the admin fee is Rp2,500.


In [ ]:
check_my_last_question = rag_chain.invoke({"input":"Apa pertanyaan terakhir saya", "chat_history":chat_history})
print(check_my_last_question["answer"])

Pertanyaan terakhir Anda adalah "How to perform a transfer payment using BNI Mobile Banking".


In [ ]:
chat_history

[HumanMessage(content='Apa saja fitur terbaru yang tersedia di BNI Mobile Banking?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The question "Apa saja fitur terbaru yang tersedia di BNI Mobile Banking?" can be rephrased as "What are the latest features available in BNI Mobile Banking?" without referencing the chat history. \n\nSince the provided context does not explicitly mention "latest features", a more accurate rephrased question would be "What features are available in BNI Mobile Banking?" \n\nThe context does provide some information about the features available in BNI Mobile Banking, such as payment and purchase of telecommunication providers, buying voucher pulsa prabayar, and paket data. However, it does not explicitly state that these are the "latest" features.\n\nTherefore, the rephrased question is: "What features are available in BNI Mobile Banking?"', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='berdasarkan pertanyaan terakhir,